# Using Q\# with Python

## Abstract

This notebook shows how to use Q# and Python together to develop and simulate quantum programs with the Quantum Development Kit.

## The `qsharp` package

We begin by importing the `qsharp` Python package. This package provides Q# interoperability for Python, as well as integration with Jupyter.

In [1]:
import qsharp

Once imported, the `qsharp` package exposes Q# functions and operations as Python objects that we can use to simulate quantum programs, estimate quantum resources, and so forth.

Let's start by defining a new Q# operation with the `%%qsharp` magic command.

In [2]:
%%qsharp

operation SayHello(name : String) : Unit {
    Message($"Hello, {name}!");
}

This creates a new Python object `SayHello` that represents our new Q# operation.

In [3]:
SayHello

<Q# callable SayHello>

By using the `.simulate` method, we can run our new operation on a quantum simulator:

In [4]:
SayHello.simulate(name="quantum world")

Hello, quantum world!


()

Inputs to your Q# operation are represented by Python keyword arguments, and outputs are returned back to the Python host.

In [5]:
%%qsharp

function Plus(x : Int, y : Int) : Int {
    return x + y;
}

In [6]:
Plus.simulate(x=3, y=5)

8

Of course, you can also use qubits in the Q# programs that you simulate from Python:

In [7]:
%%qsharp
open Microsoft.Quantum.Measurement;

operation SampleQrng() : Result {
    use q = Qubit();
    H(q);
    return MResetZ(q);
}

In [8]:
SampleQrng.simulate()

1

The `qsharp` package also provides the `compile` function, which allows for compiling Q# code from Python strings.

In [9]:
sample_qrng = qsharp.compile("""
    operation SampleQrng() : Result {
        use q = Qubit();
        H(q);
        return MResetZ(q);
    }
""")

In [10]:
sample_qrng.simulate()

1

You can also use packages with additional Q# libraries from Python using `qsharp`. To see a list of current packges, you can use `qsharp.packages`:

In [11]:
qsharp.packages

Adding package .

[('Microsoft.Quantum.Standard', LooseVersion ('0.18.2108160999')), ('Microsoft.Quantum.Standard.Visualization', LooseVersion ('0.18.2108160999'))]

By calling `qsharp.packages.add`, you can download and use new packages from nuget.org:

In [12]:
qsharp.packages.add('Microsoft.Quantum.Chemistry')

Adding package Microsoft.Quantum.Chemistry.

In [13]:
qsharp.packages

Adding package .

[('Microsoft.Quantum.Standard', LooseVersion ('0.18.2108160999')), ('Microsoft.Quantum.Standard.Visualization', LooseVersion ('0.18.2108160999')), ('Microsoft.Quantum.Chemistry', LooseVersion ('0.18.2108160999'))]

## Diagnostics

Just as with Q# standalone notebooks, you can also use diagnostics like `DumpMachine` and `DumpOperation` from Python notebooks to learn how your Q# program work and to help diagnose issues and bugs in your Q# programs.

In [14]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpPlusState() : Unit {
    use q = Qubit();
    within {
        H(q);
    } apply {
        DumpMachine();
    }
}

In [15]:
DumpPlusState.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

()

> To learn more about the diagnostics features offered by Q# and the Quantum Development Kit, check out the [visualization sample](../../diagnostics/visualization/Visualizing%20Quantum%20Programs.ipynb).

The Q# package also allows you to capture these diagnostics and manipulate them as Python objects:

In [16]:
with qsharp.capture_diagnostics() as diagnostics:
    DumpPlusState.simulate()

In [17]:
diagnostics

[{'diagnostic_kind': 'state-vector',
  'div_id': 'dump-machine-div-0f669f96-886d-4296-887f-84e9e5c6e37c',
  'qubit_ids': [0],
  'n_qubits': 1,
  'amplitudes': [{'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0},
   {'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0}]}]

Working with raw JSON for diagnostics can be somewhat inconvienent, so the `capture_diagnostics` function also supports converting diagnostics into quantum objects using the QuTiP library:

In [18]:
with qsharp.capture_diagnostics(as_qobj=True) as diagnostics:
    DumpPlusState.simulate()

In [19]:
diagnostics[0]

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

## Using other simulators

Other simulators such as the Toffoli simulator, resources estimator, and open systems simulator are available as methods on the Python representations of Q# callables. For example, you can run Q# programs on the Toffoli simulator by using the `toffoli_simulate` method:

In [20]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpOneState() : Unit {
    use q = Qubit();
    within {
        X(q);
    } apply {
        DumpMachine();
    }
}

In [21]:
DumpOneState.toffoli_simulate()

Offset  	State Data
========	==========
00000000	1


()

Similarly, the resources estimator can be used with the `estimate_resources()` method:

In [22]:
%%qsharp
operation PrepareBellPair(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    H(left);
    CNOT(left, right);
}
    
operation PrepareAndMeasureBellPair() : (Result, Result) {
    use left = Qubit();
    use right = Qubit();

    PrepareBellPair(left, right);
    return (MResetZ(left), MResetZ(right));
}

In [23]:
PrepareAndMeasureBellPair.estimate_resources()

{'CNOT': 1,
 'QubitClifford': 1,
 'R': 0,
 'Measure': 2,
 'T': 0,
 'Depth': 0,
 'Width': 2,
 'QubitCount': 2,
 'BorrowedWidth': 0}

To use the open systems simulator currently available as a preview, first enable experimental features, then use the `.simulate_noise` method:

In [24]:
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

In [25]:
DumpOneState.simulate_noise()

# of qubits,3
State data,$$ \left( \begin{matrix} 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 1 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i \end{matrix} \right) $$


()

> To learn more about using Q# and Python together, please see:
> - [`qsharp` API reference](https://docs.microsoft.com/python/qsharp-core/qsharp)
>
> To learn more about using the open systems simulator, please see:
> - [Process tomography sample](../../characterization/process-tomography/README.md)
> - [Preview documentation](https://github.com/microsoft/qsharp-runtime/blob/main/documentation/preview-simulators.md)